3.3.1. Plane cantilever beam  
135个节点  
预测位移 ${u_{x}}$ 

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from mxnet import autograd, gluon, init, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
import random
import numpy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import colors

npx.set_np()

In [ ]:
DXX=pd.read_table(r"..\Data\New_NN135_LWm_Ev_Frand_XX.txt", sep='\s+', header=None)
DYY=pd.read_table(r"..\Data\New_NN135_LWm_Ev_Frand_YY.txt", sep='\s+', header=None)

In [ ]:
NXX=np.array(DXX.values, dtype=np.float32, ctx=npx.gpu()).reshape(10000,8,1,135)

In [ ]:
YY=DYY.iloc[:,135*0:135*1]*10000

In [ ]:
n_train = 8000
train_features = NXX[:n_train,:,:,:]
test_features = NXX[n_train:,:,:,:]
train_labels = np.array(YY[:n_train].values, dtype=np.float32, ctx=npx.gpu())
test_labels = np.array(YY[n_train:].values, dtype=np.float32, ctx=npx.gpu())

In [ ]:
batch_size=256
train_iter = d2l.load_array((train_features, train_labels), batch_size)
test_iter = d2l.load_array((test_features, test_labels), batch_size)

In [ ]:
def MSE(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += ((net(X)-y)**2).sum()
        n += y.size
    return acc_sum / n

In [ ]:
net = nn.Sequential()
net.add(nn.BatchNorm(),
        nn.Conv2D(channels=8, kernel_size=3, padding=1, activation='relu'),
        nn.Dense(270, activation='relu'),
        nn.Dense(135))
net.initialize(init.Normal(sigma=0.01), ctx=npx.gpu())
loss = gluon.loss.L2Loss()

In [ ]:
train_ls, test_ls = [], []

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.00001,})

In [ ]:
old_epochs=0
new_epochs=200

for epoch in range(new_epochs):
    for X, y in train_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    train_ls.append(MSE(train_iter, net))
    test_ls.append(MSE(test_iter, net))
    if epoch % 10 ==0:
        print('epoch %d, train MSE %.6f, test MSE %.6f' % (old_epochs+epoch+1, train_ls[-1], test_ls[-1]))

In [ ]:
print(MSE(train_iter, net), MSE(test_iter, net))
plt.plot(np.arange(old_epochs+new_epochs),train_ls,label='train data')
plt.plot(np.arange(old_epochs+new_epochs),test_ls,label='test data')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.yscale('log')
plt.legend()

In [ ]:
# net.save_parameters("2Delastic_LWev_135_1__Batch14_87")
# net.load_parameters("2Delastic_LWev_135_1__Batch14_87")

In [ ]:
def plots(net,ax,iindex=0):
    if iindex==0:
        ii=random.randint(n_train,DXX.shape[0])
    else:
        ii=iindex
    xd=DXX.iloc[ii,0:135].values
    yd=DXX.iloc[ii,135:270].values

    px=NXX[ii,:,:,:]
    py=YY.iloc[ii].values
    
    nxx=px.reshape(1,8,1,135)
    nyy=net(nxx).reshape(135,)
    
    zd=py
    zn=nyy.asnumpy()

    ax.scatter3D(xd,yd,zd, color='r',label='FEM' )
    ax.scatter3D(xd,yd,zn, color='b',label='NET' )
    ax.legend()

def plotopo(net,fig,style=1,iindex=0):
    if iindex==0:
        ii=random.randint(n_train,DXX.shape[0])
    else:
        ii=iindex
    xd=DXX.iloc[ii,0:135].values
    yd=DXX.iloc[ii,135:270].values
    triang = mtri.Triangulation(xd, yd)

    px=NXX[ii,:,:,:]
    py=YY.loc[ii].values
    
    nxx=px.reshape(1,8,1,135)
    nyy=net(nxx).reshape(135,)
    ny=nyy.asnumpy()

    # py=py*(maxdisp[i]-mindisp[i])+mindisp[i]
    # ny=ny*(maxdisp[i]-mindisp[i])+mindisp[i]
    zd=py
    zn=ny
    
    vmin=min(numpy.min(zd),numpy.min(zn))
    vmax=max(numpy.max(zd),numpy.max(zn))
    norm=colors.Normalize(vmin=vmin,vmax=vmax)
    
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_aspect(1)
    if style==1:
        im=ax1.tripcolor(triang, zd, cmap="jet", shading='gouraud',norm=norm)
    elif style==2:
        # im=ax1.tricontourf(triang, zd, cmap="jet")
        im=ax1.tricontourf(triang, zd, cmap="jet",norm=norm)
    # divider = make_axes_locatable(ax1)
    # cax = divider.append_axes("right", size="6%", pad=0.15)
    # cbar = plt.colorbar(im, cax=cax)
    # cbar.set_label('$u_y$',size=10)
    # cbar.ax.tick_params(labelsize=10)
    ax1.set_title('FEM')

    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_aspect(1)
    if style==1:
        im=ax2.tripcolor(triang, zn, cmap="jet", shading='gouraud',norm=norm)
    elif style==2:
        im=ax2.tricontourf(triang, zn, cmap="jet",norm=norm)
    
    fig.subplots_adjust(right=0.9)
    position = fig.add_axes([0.92, 0.12, 0.015, 0.76])
    # cbar = plt.colorbar(im, cax=cax)
    # cbar.set_label('$u_y$',size=10)
    # cbar.ax.tick_params(labelsize=10)
    cb=fig.colorbar(im, cax=position, ax=[ax1,ax2],norm=norm, cmap="jet",)
    cb.ax.tick_params(direction='in')
    ax2.set_title('NET')

In [ ]:
fig=plt.figure(figsize=(10,8))
ax = fig.add_subplot(2, 2, 1, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 2, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 3, projection='3d');plots(net,ax)
ax = fig.add_subplot(2, 2, 4, projection='3d');plots(net,ax)

In [ ]:
for i in range(3):
    fig=plt.figure(figsize=(10,2));plotopo(net,fig,2)